In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from obspy.signal.trigger import classic_sta_lta, trigger_onset
import numpy as np

def sta_lta_graph(df, sta=500, lta=5000, thr_on=3, thr_off=1):
    if "time_rel(sec)" not in df.columns or "velocity(m/s)" not in df.columns:
        raise ValueError("Required columns 'time_rel(sec)' or 'velocity(m/s)' not found in the DataFrame")

    if len(df) < 2:
        raise ValueError("DataFrame must contain at least two rows")

    samp_rate = 1/(df["time_rel(sec)"].iloc[1] - df["time_rel(sec)"].iloc[0])
    rel_time = df["time_rel(sec)"]
    offset = int(samp_rate * 240)

    cft = classic_sta_lta(df["velocity(m/s)"], int(sta * samp_rate), int(lta * samp_rate))
    on_off = np.array(trigger_onset(cft, thr_on, thr_off))

    trig_indeces = []
    fig, ax = plt.subplots(1, 1, figsize=(12, 3))
    
    for i in range(len(on_off)):
        trigger = on_off[i]
        start_index = max(0, min(trigger[0] - offset, len(rel_time) - 1))
        end_index = min(trigger[1], len(rel_time) - 1)
        
        trig_indeces.append((start_index, end_index))
        
        ax.axvline(x=rel_time[start_index], color='red', label='Trig. On' if i == 0 else "")
        ax.axvline(x=rel_time[end_index], color='purple', label='Trig. Off' if i == 0 else "")

    ax.plot(rel_time, df["velocity(m/s)"])
    ax.set_xlim([min(rel_time), max(rel_time)])
    ax.legend()

    return trig_indeces, fig

def process_csv_files(base_path):
    directories = [
        'S12_GradeB',
        'S15_GradeA',
        'S15_GradeB',
        'S16_GradeA',
        'S16_GradeB'
    ]

    for directory in directories:
        dir_path = os.path.join(base_path, directory)
        if not os.path.exists(dir_path):
            print(f"Directory not found: {dir_path}")
            continue

        output_dir = os.path.join(dir_path, 'output')
        os.makedirs(output_dir, exist_ok=True)

        for filename in os.listdir(dir_path):
            if filename.endswith('.csv'):
                file_path = os.path.join(dir_path, filename)
                try:
                    df = pd.read_csv(file_path)
                    trig_indices, fig = sta_lta_graph(df)
                    
                    # Save the plot
                    plot_filename = f"{os.path.splitext(filename)[0]}_plot.png"
                    plot_path = os.path.join(output_dir, plot_filename)
                    fig.savefig(plot_path)
                    plt.close(fig)
                    
                    # Save trigger indices
                    indices_filename = f"{os.path.splitext(filename)[0]}_triggers.txt"
                    indices_path = os.path.join(output_dir, indices_filename)
                    with open(indices_path, 'w') as f:
                        for start, end in trig_indices:
                            f.write(f"{start},{end}\n")
                    
                    print(f"Processed {filename}")
                except Exception as e:
                    print(f"Error processing {filename}: {str(e)}")

base_path = "space_apps_2024_seismic_detection/data/lunar/test/data"
process_csv_files(base_path)